In [1]:
import csv
import json
import numpy as np
import os
import pandas as pd
import parquet
import pdb
from sklearn.metrics import r2_score
import urllib.request

In [65]:
assignment_dir = "/Applications/Anderson TA/assignment3"
parquet_path = "https://danylo-ucla.s3-us-west-2.amazonaws.com/storage/weather/jobs/transform_batch/20191012-224036_11821011c9a918e8d06e521c8606235e/score_y_solution_2356.parquet"
submitted_parquet_path = "score_y_hat.parquet"
results_file = "assignment_3_results.csv"

In [49]:
results = pd.DataFrame(columns=['Name','R2','percentile','points'])

In [50]:
df = pd.read_parquet(parquet_path)

In [51]:
true_values = df["temperature"].values

In [52]:
true_values = true_values.reshape(-1,1)

In [53]:
true_values.shape

(57576, 1)

In [54]:
submissions = os.listdir(assignment_dir)

In [55]:
#calculate R2
count = 0
for item in submissions:
    if(item=='.DS_Store'):
        continue
    temp = item.split('_')
    name = temp[0]
    submitted_values = None
    if(os.path.exists(os.path.join(assignment_dir,item,submitted_parquet_path))):
        submitted_values = pd.read_parquet(os.path.join(assignment_dir,item,submitted_parquet_path))
    submitted_values = submitted_values.values
    r2 = r2_score(true_values, submitted_values)
    results.loc[count] = [name, r2,0,0]
    count = count + 1
    

In [56]:
results

,Name,R2,percentile,points
0,"RUAN, TIANYI",0.928373,0,0
1,"MALAR ELANGOVAN, EZHILVEL",0.880162,0,0
2,"HEBBAR, DIVAKAR",0.914603,0,0
3,"MAKAROVA, MARINA",0.912029,0,0
4,"JIANG, LING",0.903800,0,0
5,"MILLWATER, ERICA",0.914214,0,0
6,"VOLETI, SRI VAMSI",0.889404,0,0
7,"SONG, KAEUN",0.825536,0,0
8,"SUN, SIYANG",0.868690,0,0
9,"MATHEW, SERIN",0.910084,0,0


In [57]:
results.loc[results['R2'] > 0.85, 'points'] = 10

In [58]:
results

,Name,R2,percentile,points
0,"RUAN, TIANYI",0.928373,0,10
1,"MALAR ELANGOVAN, EZHILVEL",0.880162,0,10
2,"HEBBAR, DIVAKAR",0.914603,0,10
3,"MAKAROVA, MARINA",0.912029,0,10
4,"JIANG, LING",0.903800,0,10
5,"MILLWATER, ERICA",0.914214,0,10
6,"VOLETI, SRI VAMSI",0.889404,0,10
7,"SONG, KAEUN",0.825536,0,0
8,"SUN, SIYANG",0.868690,0,10
9,"MATHEW, SERIN",0.910084,0,10


In [59]:
tempresults = results[~results.index.isin([7])]

In [61]:
percentiles = np.array(tempresults.R2.quantile(np.linspace(0.1, 1.1, 10, 0)))

In [62]:
percentiles

array([0.87098455, 0.88385874, 0.89765743, 0.90367192, 0.9066753 ,
       0.91047282, 0.91290303, 0.91444759, 0.91993569, 0.92837347])

In [63]:
#calculate percentile and add points
for index, row in results.iterrows():
    r2 = row['R2']
    if r2==percentiles[-1]:
        upper_bound = r2
        val = 10
    elif r2<percentiles[0]:
        upper_bound = percentiles[0]
        val = 1
    else:
        i=0
        while(i<len(percentiles)):
            if(r2<=percentiles[i]):
                upper_bound = percentiles[i]
                val = i+1
                break
            i=i+1
    results.at[index,'percentile'] = val
    results.at[index,'points'] = row['points'] + val
    print(row['R2'], upper_bound, val)

0.9283734697025172 0.9283734697025172 10
0.880162148273915 0.8838587402085328 2
0.9146031859452088 0.9199356906233047 9
0.9120289157279614 0.9129030314961305 7
0.9037999149474616 0.9066752950680002 5
0.9142142051483841 0.914447593626479 8
0.8894036281104595 0.89765743020633 3
0.8255358857076212 0.8709845518276426 1
0.8686901527160744 0.8709845518276426 1
0.9100837899427933 0.9104728150998269 6
0.8515345961249683 0.8709845518276426 1
0.9066752950680002 0.9066752950680002 5
0.9031599649369104 0.9036719249453513 4
0.9212688167928286 0.9283734697025172 10


In [64]:
results

,Name,R2,percentile,points
0,"RUAN, TIANYI",0.928373,10,20
1,"MALAR ELANGOVAN, EZHILVEL",0.880162,2,12
2,"HEBBAR, DIVAKAR",0.914603,9,19
3,"MAKAROVA, MARINA",0.912029,7,17
4,"JIANG, LING",0.903800,5,15
5,"MILLWATER, ERICA",0.914214,8,18
6,"VOLETI, SRI VAMSI",0.889404,3,13
7,"SONG, KAEUN",0.825536,1,1
8,"SUN, SIYANG",0.868690,1,11
9,"MATHEW, SERIN",0.910084,6,16


In [66]:
results.to_csv(results_file,index=False,columns=results.columns)

In [ ]:
#end